# Barbuilder
## An Interactive Cocktail Ingredient Recommendation System

### Set Up System

In [1]:
# imports

import base64
from bitarray import bitarray
import csv
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import plotly.express as px
import urllib
from wordcloud import WordCloud

In [2]:
# locations for file imports

processed_data = Path.cwd() / 'processed' / 'cosine_matrix.pickle'
ingredient_data = Path.cwd() / 'processed' / 'ingredient_dict.pickle'
recipe_data = Path.cwd() / 'processed' / 'recipe_dict.pickle'

In [3]:
# load Data (Generate data if not found, can take several minutes on first run to download raw files)

# check if data already generated. If not, run data_processing.ipynb.
try:
    open(processed_data, mode='r', encoding='utf-8')
    open(ingredient_data, mode='r', encoding='utf-8')
except FileNotFoundError as err:
    %run ./data_processing.ipynb

#  load the data files
with open(processed_data,'rb') as modelFile:
     df_sims = pickle.load(modelFile)
with open(ingredient_data,'rb') as modelFile:
     df_ingredients = pickle.load(modelFile)
with open(recipe_data,'rb') as modelFile:
     recipe_dict = pickle.load(modelFile)

In [4]:
# prepare data for different uses

# clean up recipe_dict (remove empty strings)  {recipe: [ingredients]}
for r in recipe_dict.keys():
    filter_object = filter(lambda x: x != "", recipe_dict[r])
    without_empty_strings = list(filter_object)
    recipe_dict[r] = without_empty_strings

# calculate and sort a dataframe of recipes / ingredient
recipes_per_ingredient = df_ingredients.sum(axis=1).sort_values(ascending=False).copy()

# dictionary of all recipes in each ingredient {ingredient: [recipes]}
ingredient_dict = {} 
for ingredient in df_ingredients.index:
    ingredient_dict[ingredient] = df_ingredients.loc[ingredient].index[df_ingredients.loc[ingredient] > 0]

In [24]:
df_sims.head()

,vodka,gin,rum,tequila,scotch,absolut kurant,absolut peppar,absolut vodka,advocaat,agave syrup,...,whisky,pomegranate juice,watermelon,rosemary syrup,figs,thyme,blood orange,amaro montenegro,ruby port,rose
vodka,1.000000,0.089148,0.121218,0.129099,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.101015,0.101015,0.101015,0.101015,0.101015,0.0
gin,0.089148,1.000000,0.058835,0.089514,0.0,0.0,0.0,0.043853,0.0,0.000000,...,0.0,0.0,0.000000,0.069338,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
rum,0.121218,0.058835,1.000000,0.073030,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
tequila,0.129099,0.089514,0.073030,1.000000,0.0,0.0,0.0,0.000000,0.0,0.210819,...,0.0,0.0,0.182574,0.129099,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
scotch,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


#### Generate Graphics

In [5]:
# bar graph of recipes / ingredient
filtered_ingred = recipes_per_ingredient >= 10
x_val = recipes_per_ingredient.index[filtered_ingred] 
y_val = recipes_per_ingredient[filtered_ingred] 
graph_recipes_per_ingredient = px.bar(x=x_val, y=y_val, width=1000, range_x=[-1,20], title='Recipes / Ingredient (Minimum 10)', 
            labels={'x':'Ingredient (click/drag ingredients to move graph sideways)', 'y':'Total Recipes'})
# graph_recipes_per_ingredient.show()

In [6]:
# heatmap with hover
 #click/drag to zoom. double click to zoom out
heatmap_ingredients = px.imshow(df_sims, color_continuous_scale='electric', aspect="equal", height=1000, title='Ingredient Similarities. Click/drag to zoom. Double-click to zoom out.')
# heatmap_ingredients.show()
out_heat = widgets.Output()
with out_heat:
    heatmap_ingredients.show()

In [7]:
# wordcloud of ingredient frequencies
wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(recipes_per_ingredient)


plt.figure(figsize=(16,8))
plt.imshow(wordcloud)
plt.axis("off")
buf = io.BytesIO()
plt.title(label='Ingredient Frequency Word Cloud', loc='center')
plt.savefig(buf, format='png')
buf.seek(0)
string = base64.b64encode(buf.read())

uri = 'data:image/png;base64,' + urllib.parse.quote(string)
plt.close()
wordcloud_ingredients_html = '<img src = "%s"/>' % uri

### Set up Data Analysis Window

In [8]:
# data_analysis_window --- data analysis window

#TITLE
title = 'Data Analysis'

#Total Recipes
total_recipes = str(len(recipe_dict))

#Total Ingredients
total_ingredients = str(len(ingredient_dict))

#Mean Ingredients/Recipes
mean_ingredients_per_recipe = sum([len(recipe_dict[x]) for x in recipe_dict.keys()]) / len(recipe_dict)

#Mean Recipes/Ingredient
mean_recipes_per_ingredient = recipes_per_ingredient.mean()

data_html = widgets.HTML('<style> ' +
                        'table { ' +
                        'font-family: arial, sans-serif;' +
                        ' border-collapse: collapse;' +
                        ' width: 80%;}' +
                        'td, th {' +
                        'border: 1px solid #dddddd;' +
                        ' text-align: left;' +
                        ' padding: 8px;' +
                        '}' +
                        ' tr:nth-child(even) {' +
                        ' background-color: #dddddd;' +
                        ' }' +
                        '</style>' + 
                        '<h1>' + title + '</h1><br><br>' +
                        '<table>' +
                        '<tr><td>Total Recipes: </td><td>' + total_recipes + '</td></tr>' +
                        '<tr><td>Total Ingredients: </td><td>' + total_ingredients + '</td></tr>' +
                        '<tr><td>Mean Ingredients / Recipe: </td><td>' + "%.2f" % mean_ingredients_per_recipe + '</td></tr>' +
                        '<tr><td>Mean Recipes / Ingredient: </td><td>' + "%.2f" % mean_recipes_per_ingredient + '</td></tr>' +
                        '</table>')
    

data_analysis_window = data_html

In [9]:
# recipe_window --- ingredient exploration window

# list of all recipes for a selected ingredient. Runs off of dict(recipes).
# drop box of ingredients
sorted_ingredients = [x.title() for x in sorted(ingredient_dict.keys())]
ingredient_selector = widgets.Dropdown(options=sorted_ingredients)

# list of all recipes in that ingredient
recipe_list_box = widgets.Select(rows=20)

# list of all ingredients in selected recipe
list_title = widgets.HTML('Ingredients for selected drink:')
drink_ingredients_list = widgets.Select(rows=20)

def update_recipe_list(*args):
    recipe_list_box.options = ingredient_dict[ingredient_selector.value.lower()]
def update_ingredients_list(*args):
    drink_ingredients_list.options = recipe_dict[recipe_list_box.value]

# update recipe list when an ingredient is chosen
ingredient_selector.observe(update_recipe_list, names='value')
# update ingredient list when a recipe is chosen
recipe_list_box.observe(update_ingredients_list, names='value')

# instructions for window
instructions = widgets.HTML('<h3>Choose an ingredient to see which recipes it is used in.<br>' +
                           'Choose a recipe to see which ingredients it uses.')

# arrange and display widgets
left_side = widgets.VBox([ingredient_selector, recipe_list_box])
right_side = widgets.VBox([list_title, drink_ingredients_list])
bottom_pane = widgets.HBox([left_side, right_side])
recipe_window = widgets.VBox([instructions, bottom_pane])

In [10]:
# bar_graph_window --- bar graph of recipes / ingredient
bar_graph_out = widgets.Output()
with bar_graph_out:
    graph_recipes_per_ingredient.show()
bar_graph_window = widgets.VBox([bar_graph_out])

In [11]:
# word_cloud_window --- word cloud of ingredient frequency
word_cloud_window = widgets.HTML(wordcloud_ingredients_html)

# Data Analysis (Descriptive Methods)

In [12]:
# assemble data analysis view and display

top_window = widgets.HBox([data_analysis_window, recipe_window])
data_analysis_view = widgets.VBox([top_window, bar_graph_window, word_cloud_window, out_heat], layout=widgets.Layout(padding='50px', width='80%'))
data_analysis_view

# Set up recommendation dashboard

In [13]:
# Recommendation Algorithm

# Returns sorted recommendations based on a list of owned ingredients
def get_recommendation(c):
    # uses df_sims, previously calculated cosine similarity matrix
    if len(c) == 0: return df_sims.sum(axis=0).sort_values(ascending=False)
        
    # filter down to customer list c
    filtered_df = df_sims.filter(items=c, axis=0)
    
    # sum up columns to get total recommendations
    summed_df = filtered_df.sum(axis=0)
    
    # delete any rows in the customer list c, don't recommend what they already have!
    result = summed_df.drop(c).sort_values(ascending=False)

    return result

In [14]:
# Heat Map Example
heat_map = plt.imshow(df_sims, cmap='hot', interpolation='nearest')
out1 = widgets.Output()
with out1:
    plt.show()
    #  display(heat_map.figure)

In [15]:
# create data objects to drive each window
sorted_ingredients = [x.title() for x in sorted(ingredient_dict.keys())] # for ingredient list
customer_ingredients = [] # for customer list
recommendations = [] # for recommendation screen

In [16]:
# interaction functions for recommender system

def generate_customer_list():
    customer_list.options = [x.title() for x in sorted(customer_ingredients)]
    ingredient_count.value = str(len(customer_ingredients))
    generate_recommendation_list()

def build_recommendation_strings(r):
    str_len = 40 # change this to modify final string length
    result = []
    for x in r.index:
        i = len(x)
        score = "%.2f" % r[x]
        mid = str_len - i - len(score)
        result.append(x.title() + ' ' + '-'*(mid-2) + ' ' + score)   
    return result

def unbuild_recommendation_string(s):
    return s[:s.find('-')-1]

def generate_recommendation_list():
    r = get_recommendation(customer_ingredients)
    recommendation_list.options = build_recommendation_strings(r)
    
def search_box_typing(*args):
    unset_observes()
    ingredient_list.options = [s.title() for s in sorted_ingredients if search_box.value.lower() in s.lower()]
    set_observes()

def click_ingredient_list(*args):
    unset_observes()
    search_box.value = ''
    clicked = ingredient_list.value
    if ~len(clicked) > 0: return
    customer_ingredients.append(clicked.lower())
    sorted_ingredients.remove(clicked.title())
    ingredient_list.options = sorted_ingredients
    set_observes()

def click_customer_list(*args):
    unset_observes()
    search_box.value = ''
    clicked = customer_list.value
    if ~len(clicked) > 0: return
    sorted_ingredients.append(clicked.title())
    sorted_ingredients.sort()
    customer_ingredients.remove(clicked.lower())
    ingredient_list.options = sorted_ingredients
    set_observes()

def click_recommendation_list(*args):
    unset_observes()
    search_box.value = ''
    clicked = recommendation_list.value
    if ~len(clicked) > 0: return
    clicked = unbuild_recommendation_string(clicked)
    customer_ingredients.append(clicked.lower())
    sorted_ingredients.remove(clicked.title())
    ingredient_list.options = sorted_ingredients
    set_observes()

def set_observes():
    generate_customer_list()
    customer_list.value=None
    ingredient_list.value=None
    recommendation_list.value=None
    search_box.observe(search_box_typing, names='value')
    ingredient_list.observe(click_ingredient_list, names='value')
    customer_list.observe(click_customer_list, names='value')
    recommendation_list.observe(click_recommendation_list, names='value')

def unset_observes():
    search_box.unobserve(search_box_typing, names='value')
    ingredient_list.unobserve(click_ingredient_list, names='value')
    customer_list.unobserve(click_customer_list, names='value')
    recommendation_list.unobserve(click_recommendation_list, names='value')

In [17]:
#basic list box layout
list_box_layout = widgets.Layout(height='400px')

# title bar
top_bar = widgets.HTML(value="<center><h1>Barbuilder</h1></n><h3>Drink Ingredient Recommendation System</h3></n>" + 
                       "<h3>This application will recommend cocktail ingredients that work well together in recipes!</h3>",)

# set up ingredients window
# search_box
search_label = widgets.Label(value='Search:')
search_box = widgets.Text(disabled=False,)

# ingredient list
ingredient_list = widgets.Select(options=sorted_ingredients, layout=list_box_layout)
ingredient_list.value = None

# assemble ingredient window
ingredient_window = widgets.VBox([search_label, search_box, ingredient_list])

# set up customer list window
customer_list_header = widgets.Label(value='Customer\'s Ingredients:')
hidden_box = widgets.Text(layout=widgets.Layout(visibility='hidden'))
customer_list = widgets.Select(options=customer_ingredients, layout=list_box_layout)

# instruction text
ingredient_count = widgets.Text(description='Ingredients: ', value='0', disabled=True, layout=widgets.Layout(width='200px'))

# assemble customer list window
customer_list_window = widgets.VBox(children=[customer_list_header, hidden_box, customer_list, ingredient_count])

# set up recommendation window
recommendation_list_header = widgets.Label(value='Purchase Recommendations / Recommendation Score:')
hidden_box = widgets.Text(layout=widgets.Layout(visibility='hidden'))
recommendation_list = widgets.Select(options=recommendations, layout=list_box_layout)

# assemble recommendation window
recommendation_window = widgets.VBox([recommendation_list_header, hidden_box, recommendation_list])

# assemble windows into main window
main_window = widgets.HBox([ingredient_window, customer_list_window, recommendation_window],layout=widgets.Layout(grid_gap='50px'))

# set up anything that goes along the bottom of the UI
bottom_bar = widgets.Label(value='Instructions:  Click ingredients in list to add to what you own ' + 
                           '(Customer\'s ingredients as well as recommendations). Click to remove ingredients from the Customer list.',)

# roll it all into a GUI
gui = widgets.VBox([top_bar, main_window, bottom_bar], layout=widgets.Layout(display='flex', flex_flow='column', align_items='center', width='80%'))

# set observe methods
set_observes()

# Recommendation Dashboard

In [18]:
# load dashboard
gui